In [1]:
from mmdet.apis import init_detector
import mmcv
from mmcv import Config


import copy
import os.path as osp

import numpy as np

from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset

from mmdet.apis import set_random_seed


import json

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import glob as _glob
import os
    
def glob(dir, pats, recursive=False):  # faster than match, python3 only
    pats = pats if isinstance(pats, (list, tuple)) else [pats]
    matches = []
    for pat in pats:
        matches += _glob.glob(os.path.join(dir, pat), recursive=recursive)
    return matches


In [3]:
#### load_annotations에서 뒤의 변수 받는거 custom dataset 에서는 이름을 바꿔도 되지만 아래에
#### configuration에서는 무조건 변수명을 ann_file로 받아야함
@DATASETS.register_module()
class Drive_dataset(CustomDataset):
    CLASSES=('car','bus','truck', 'special vehicle', 'motorcycle','bicycle','personal mobility','person','Traffic_light', 'Traffic_sign')


    def load_annotations(self, ann_fol):
        
        CLASSES_dict = {'car' : 0 , 'bus' : 1, 'truck' : 2, 'special vehicle' : 3, 'motorcycle' : 4,'bicycle' : 5 ,'personal mobility' : 6 
                        ,'person' : 7 ,'Traffic_light' : 8, 'Traffic_sign' : 9}
        
        cat2label = {k: i for i, k in enumerate(self.CLASSES)}
        
        data_infos = []
        
        ls = glob(ann_fol,'*',True)
        
        for idx,an in enumerate(ls):
            json_data = {}
            with open(an, "r") as json_file:
                json_data = json.load(json_file)
                
            ansplit = an.split('/')
            
            filename = ansplit[0] + '/' + ansplit[1] + '/' + 'images'+'/'+ json_data['image_name']
            
            width, height = json_data['image_size']

            data_info = dict(filename=filename, width=width, height=height)

            gt_bboxes = []
            gt_labels = []

            for ann_data in json_data['Annotation']:
                gt_labels.append(CLASSES_dict[ann_data['class_name']])
                gt_bboxes.append(ann_data['data'])


            data_anno = dict(
                    bboxes=np.array(gt_bboxes, dtype=np.float32).reshape(-1, 4),
                    labels=np.array(gt_labels, dtype=np.long))


            data_info.update(ann=data_anno)
            data_infos.append(data_info)
            
            if idx!=0 and idx%20000==0:
                print(str(idx)+'/'+str(len(ls))+' load annotations END!')
            
        
        
        return data_infos

In [4]:
## 추가수정 기존 받았던 pretrain과 매칭되는 config로 수정 
cfg = Config.fromfile('UniverseNet/configs/waymo_open/universenet50_2008_fp16_4x4_mstrain_640_1280_1x_waymo_open_f0.py') 

In [5]:
print(f'Config:\n{cfg.pretty_text}')

Config:
pretrained_ckpt = 'https://github.com/shinya7y/weights/releases/download/v1.0.2/res2net50_v1b_26w_4s-3cf99910_mmdetv2-92ed3313.pth'
model = dict(
    type='GFL',
    backbone=dict(
        type='Res2Net',
        depth=50,
        scales=4,
        base_width=26,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='SyncBN', requires_grad=True),
        norm_eval=False,
        style='pytorch',
        dcn=dict(type='DCN', deform_groups=1, fallback_on_stride=False),
        stage_with_dcn=(False, False, False, True),
        init_cfg=dict(
            type='Pretrained',
            checkpoint=
            'https://github.com/shinya7y/weights/releases/download/v1.0.2/res2net50_v1b_26w_4s-3cf99910_mmdetv2-92ed3313.pth'
        )),
    neck=[
        dict(
            type='FPN',
            in_channels=[256, 512, 1024, 2048],
            out_channels=256,
            start_level=1,
            add_extra_convs='on_output',
   

In [11]:
## 추가 및 수정 ## 
cfg.dataset_type  = 'Drive_dataset'
cfg.data_root = ''

## single GPU 이기 때문에 syncBN 이 아닌 BN으로 수정)
cfg.model.backbone.norm_cfg=dict(type='BN', requires_grad=True)

## Validation pipeline에 train pipeline 적용하기 위해서 구성 
cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='Resize',
        img_scale=(1920, 1200),
        multiscale_mode='range',
        keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.0),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
    
]

### test pipeline 나중에 test진행에 사용할 거 실제 validation은 위의 pipeline 으로 진행
cfg.test_pipeline = [
    ### TSET때 사용할 test time augmentation용 pipeline
    dict(type='LoadImageFromFile'),
    dict(
                type='MultiScaleFlipAug',
                img_scale=(1920, 1200),
                flip=False,
                transforms=[
                    dict(type='Resize', keep_ratio=True),
                    dict(type='RandomFlip'),
                    dict(
                        type='Normalize',
                        mean=[123.675, 116.28, 103.53],
                        std=[58.395, 57.12, 57.375],
                        to_rgb=True),
                    dict(type='Pad', size_divisor=32),
                    dict(type='ImageToTensor', keys=['img']),
                      dict(type='Collect', keys=['img'])
                ])
]


cfg.data=dict(
    samples_per_gpu=10, # batchsize
    workers_per_gpu=12, # batch를 불러오기 위한 작업 thread 갯수 
    # train dataset 
    train=dict(
        type=cfg.dataset_type,
        ann_file='2DBB/training/labels/',
        pipeline=cfg.train_pipeline),
    # validation dataset  
    val=dict(
        type=cfg.dataset_type,
        ann_file='2DBB/validation/labels',
        pipeline=cfg.test_pipeline),
    test=dict(
        type=cfg.dataset_type,
        ann_file='2DBB/test/labels/',
        pipeline=cfg.test_pipeline),)

## class 갯수 
cfg.model.bbox_head.num_classes=10

## GPU 학습 진행을 위한 device 선언
cfg.device='cuda'

## weight 와 학습 log 저장 위치 
cfg.work_dir = 'checkpoints_Best_ver1'

## log interval
cfg.log_config.interval = 8000 #iteration 단위

cfg.seed = 2024

## seed 고정 진행
set_random_seed(cfg.seed, deterministic=False)

cfg.workflow = [('train', 1), ('val',1)]

cfg.evaluation = dict(interval=1, metric='mAP')

### coco dataset으로 pretrain된 weight load 할 path
cfg.load_from = 'universenet50_2008_fp16_4x4_mstrain_480_960_2x_coco_20200815_epoch_24-81356447.pth'

### epoch 선언
cfg.runner = dict(type='EpochBasedRunner', max_epochs=50)

### 사용할 GPU 선언
cfg.gpu_ids = range(1)

In [12]:
print(f'Config:\n{cfg.pretty_text}')

Config:
pretrained_ckpt = 'https://github.com/shinya7y/weights/releases/download/v1.0.2/res2net50_v1b_26w_4s-3cf99910_mmdetv2-92ed3313.pth'
model = dict(
    type='GFL',
    backbone=dict(
        type='Res2Net',
        depth=50,
        scales=4,
        base_width=26,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=False,
        style='pytorch',
        dcn=dict(type='DCN', deform_groups=1, fallback_on_stride=False),
        stage_with_dcn=(False, False, False, True),
        init_cfg=dict(
            type='Pretrained',
            checkpoint=
            'https://github.com/shinya7y/weights/releases/download/v1.0.2/res2net50_v1b_26w_4s-3cf99910_mmdetv2-92ed3313.pth'
        )),
    neck=[
        dict(
            type='FPN',
            in_channels=[256, 512, 1024, 2048],
            out_channels=256,
            start_level=1,
            add_extra_convs='on_output',
       

In [13]:
from mmdet.datasets import build_dataset, build_dataloader

In [14]:
test_dataloader_default_args = dict(
    samples_per_gpu=1, workers_per_gpu=2, dist=False, shuffle=False)

dataset = build_dataset(cfg.data.test)

test_loader_cfg = {
        **test_dataloader_default_args,
        **cfg.data.get('test_dataloader', {})
    }
# build the dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(dataset, **test_loader_cfg)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/root/UniverseNet/mmdet/datasets/custom.py:186: UserWarning: CustomDataset does not support filtering empty gt images.
  'CustomDataset does not support filtering empty gt images.')


In [15]:
from mmdet.models import build_detector
from mmcv.runner import (get_dist_info, init_dist, load_checkpoint,
                         wrap_fp16_model)

In [16]:
checkpoint_path = 'checkpoints_Best_ver1/epoch_50.pth'

# build the model and load checkpoint
cfg.model.train_cfg = None
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
fp16_cfg = cfg.get('fp16', None)
if fp16_cfg is not None:
    wrap_fp16_model(model)
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')
model.CLASSES = dataset.CLASSES

/root/UniverseNet/mmdet/models/dense_heads/anchor_head.py:116: UserWarning: DeprecationWarning: `num_anchors` is deprecated, for consistency or also use `num_base_priors` instead
  warnings.warn('DeprecationWarning: `num_anchors` is deprecated, '


load checkpoint from local path: checkpoints_Best_ver1/epoch_50.pth


2024-08-14 03:28:08,156 - root - INFO - DeformConv2dPack backbone.layer4.0.convs.0 is upgraded to version 2.
2024-08-14 03:28:08,157 - root - INFO - DeformConv2dPack backbone.layer4.0.convs.1 is upgraded to version 2.
2024-08-14 03:28:08,158 - root - INFO - DeformConv2dPack backbone.layer4.0.convs.2 is upgraded to version 2.
2024-08-14 03:28:08,160 - root - INFO - DeformConv2dPack backbone.layer4.1.convs.0 is upgraded to version 2.
2024-08-14 03:28:08,161 - root - INFO - DeformConv2dPack backbone.layer4.1.convs.1 is upgraded to version 2.
2024-08-14 03:28:08,162 - root - INFO - DeformConv2dPack backbone.layer4.1.convs.2 is upgraded to version 2.
2024-08-14 03:28:08,164 - root - INFO - DeformConv2dPack backbone.layer4.2.convs.0 is upgraded to version 2.
2024-08-14 03:28:08,165 - root - INFO - DeformConv2dPack backbone.layer4.2.convs.1 is upgraded to version 2.
2024-08-14 03:28:08,165 - root - INFO - DeformConv2dPack backbone.layer4.2.convs.2 is upgraded to version 2.


In [17]:
from mmdet.utils import (build_ddp, build_dp, compat_cfg, get_device,
                         replace_cfg_vals, setup_multi_processes,
                         update_data_root)

from mmdet.apis import single_gpu_test ## 4070 super 단독 환경

In [18]:
model = build_dp(model, cfg.device, device_ids=cfg.gpu_ids)
outputs = single_gpu_test(model, data_loader)

[                               ] 2/10000, 0.5 task/s, elapsed: 4s, ETA: 18559s

/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448265233/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[>>>>>>>>>>>>>>>>>>>>>>>>] 10000/10000, 15.6 task/s, elapsed: 639s, ETA:     0s

In [19]:
# for oss in outputs:
#     for o in oss:
#         print(len(o))

In [20]:
mAP_output = dataset.evaluate(outputs,metric='mAP',iou_thr=0.5)


---------------iou_thr: 0.5---------------

+-------------------+-------+--------+--------+-------+
| class             | gts   | dets   | recall | ap    |
+-------------------+-------+--------+--------+-------+
| car               | 32446 | 205634 | 0.988  | 0.967 |
| bus               | 947   | 8736   | 0.983  | 0.955 |
| truck             | 5759  | 62610  | 0.980  | 0.945 |
| special vehicle   | 194   | 4903   | 0.969  | 0.896 |
| motorcycle        | 224   | 4469   | 0.951  | 0.913 |
| bicycle           | 65    | 1193   | 1.000  | 0.977 |
| personal mobility | 34    | 1265   | 0.912  | 0.893 |
| person            | 7221  | 51878  | 0.938  | 0.871 |
| Traffic_light     | 3624  | 21844  | 0.977  | 0.935 |
| Traffic_sign      | 2982  | 27370  | 0.974  | 0.935 |
+-------------------+-------+--------+--------+-------+
| mAP               |       |        |        | 0.929 |
+-------------------+-------+--------+--------+-------+
